In [9]:
%%writefile fcnet.cu

Overwriting fcnet.cu


In [16]:
%%writefile fcnet.cu
#include <cudnn.h>
#include <cuda_runtime.h>
#include <curand.h>
#include <cstdlib>
#include <ctime>
#include <iostream>
#include <cstdio> // Include for printf

#define CHECK_CUDA(func) { \
    cudaError_t status = (func); \
    if (status != cudaSuccess) { \
        std::cerr << "CUDA Error at " << __FILE__ << ":" << __LINE__ << " - " << cudaGetErrorString(status) << std::endl; \
        exit(EXIT_FAILURE); \
    } \
}

#define CHECK_CUDNN(func) { \
    cudnnStatus_t status = (func); \
    if (status != CUDNN_STATUS_SUCCESS) { \
        std::cerr << "cuDNN Error at " << __FILE__ << ":" << __LINE__ << " - " << cudnnGetErrorString(status) << std::endl; \
        exit(EXIT_FAILURE); \
    } \
}

// Network parameters
const int input_size = 1000;
const int hidden_size = 512;
const int output_size = 10;
const int batch_size = 64;
//const float learning_rate = 0.001f;
const int epochs = 10;

// Helper function to initialize weights
void initialize_weights(float* weights, int size) {
    curandGenerator_t gen;
    curandCreateGenerator(&gen, CURAND_RNG_PSEUDO_DEFAULT);
    curandSetPseudoRandomGeneratorSeed(gen, time(NULL));
    curandGenerateUniform(gen, weights, size);
    curandDestroyGenerator(gen);
}

int main() {
    printf("Initializing CUDA and cuDNN...\n");
    // Initialize CUDA and cuDNN
    cudnnHandle_t cudnn;
    CHECK_CUDNN(cudnnCreate(&cudnn));
    printf("CUDA and cuDNN initialized.\n");

    // Create tensor descriptors
    printf("Creating tensor descriptors...\n");
    cudnnTensorDescriptor_t input_desc, hidden1_desc, hidden2_desc, output_desc;
    CHECK_CUDNN(cudnnCreateTensorDescriptor(&input_desc));
    CHECK_CUDNN(cudnnCreateTensorDescriptor(&hidden1_desc));
    CHECK_CUDNN(cudnnCreateTensorDescriptor(&hidden2_desc));
    CHECK_CUDNN(cudnnCreateTensorDescriptor(&output_desc));
    printf("Tensor descriptors created.\n");

    // Set tensor dimensions (NCHW format)
    printf("Setting tensor dimensions...\n");
    CHECK_CUDNN(cudnnSetTensor4dDescriptor(input_desc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT,
                                          batch_size, input_size, 1, 1));
    CHECK_CUDNN(cudnnSetTensor4dDescriptor(hidden1_desc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT,
                                          batch_size, hidden_size, 1, 1));
    CHECK_CUDNN(cudnnSetTensor4dDescriptor(hidden2_desc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT,
                                          batch_size, hidden_size, 1, 1));
    CHECK_CUDNN(cudnnSetTensor4dDescriptor(output_desc, CUDNN_TENSOR_NCHW, CUDNN_DATA_FLOAT,
                                          batch_size, output_size, 1, 1));
    printf("Tensor dimensions set.\n");

    // Create filter descriptors (weights)
    printf("Creating filter descriptors...\n");
    cudnnFilterDescriptor_t fc1_w_desc, fc2_w_desc, fc3_w_desc;
    CHECK_CUDNN(cudnnCreateFilterDescriptor(&fc1_w_desc));
    CHECK_CUDNN(cudnnSetFilter4dDescriptor(fc1_w_desc, CUDNN_DATA_FLOAT, CUDNN_TENSOR_NCHW,
                                          hidden_size, input_size, 1, 1));

    CHECK_CUDNN(cudnnCreateFilterDescriptor(&fc2_w_desc));
    CHECK_CUDNN(cudnnSetFilter4dDescriptor(fc2_w_desc, CUDNN_DATA_FLOAT, CUDNN_TENSOR_NCHW,
                                          hidden_size, hidden_size, 1, 1));

    CHECK_CUDNN(cudnnCreateFilterDescriptor(&fc3_w_desc));
    CHECK_CUDNN(cudnnSetFilter4dDescriptor(fc3_w_desc, CUDNN_DATA_FLOAT, CUDNN_TENSOR_NCHW,
                                          output_size, hidden_size, 1, 1));
    printf("Filter descriptors created.\n");

    // Create activation descriptor (ReLU)
    printf("Creating activation descriptor...\n");
    cudnnActivationDescriptor_t relu_desc;
    CHECK_CUDNN(cudnnCreateActivationDescriptor(&relu_desc));
    CHECK_CUDNN(cudnnSetActivationDescriptor(relu_desc, CUDNN_ACTIVATION_RELU,
                                             CUDNN_NOT_PROPAGATE_NAN, 0.0));
    printf("Activation descriptor created.\n");

    // Allocate device memory
    printf("Allocating device memory...\n");
    float *d_input, *d_labels, *d_output;
    CHECK_CUDA(cudaMalloc(&d_input, batch_size * input_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_labels, batch_size * output_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_output, batch_size * output_size * sizeof(float)));
    printf("Device memory allocated.\n");

    // Initialize weights and biases
    printf("Initializing weights and biases...\n");
    float *d_w1, *d_b1, *d_w2, *d_b2, *d_w3, *d_b3;
    CHECK_CUDA(cudaMalloc(&d_w1, hidden_size * input_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b1, hidden_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_w2, hidden_size * hidden_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b2, hidden_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_w3, output_size * hidden_size * sizeof(float)));
    CHECK_CUDA(cudaMalloc(&d_b3, output_size * sizeof(float)));

    initialize_weights(d_w1, hidden_size * input_size);
    initialize_weights(d_w2, hidden_size * hidden_size);
    initialize_weights(d_w3, output_size * hidden_size);
    CHECK_CUDA(cudaMemset(d_b1, 0, hidden_size * sizeof(float)));
    CHECK_CUDA(cudaMemset(d_b2, 0, hidden_size * sizeof(float)));
    CHECK_CUDA(cudaMemset(d_b3, 0, output_size * sizeof(float)));
    printf("Weights and biases initialized.\n");

    // Generate dummy data
    printf("Generating dummy data...\n");
    initialize_weights(d_input, batch_size * input_size);
    initialize_weights(d_labels, batch_size * output_size);
    printf("Dummy data generated.\n");

    // Create convolution descriptors
    printf("Creating convolution descriptor...\n");
    cudnnConvolutionDescriptor_t conv_desc;
    CHECK_CUDNN(cudnnCreateConvolutionDescriptor(&conv_desc));
    CHECK_CUDNN(cudnnSetConvolution2dDescriptor(conv_desc, 0, 0, 1, 1, 1, 1,
                                               CUDNN_CROSS_CORRELATION, CUDNN_DATA_FLOAT));
    printf("Convolution descriptor created.\n");

    // Training loop
    printf("Starting training loop for %d epochs...\n", epochs);
    for (int epoch = 0; epoch < epochs; ++epoch) {
        printf("Epoch %d/%d\n", epoch + 1, epochs);
        // Forward pass
        float alpha = 1.0f, beta = 0.0f;
        float* d_hidden1, *d_hidden2;
        CHECK_CUDA(cudaMalloc(&d_hidden1, batch_size * hidden_size * sizeof(float)));
        CHECK_CUDA(cudaMalloc(&d_hidden2, batch_size * hidden_size * sizeof(float)));

        // Layer 1: Input -> Hidden1
        CHECK_CUDNN(cudnnConvolutionForward(cudnn, &alpha,
                                           input_desc, d_input,
                                           fc1_w_desc, d_w1,
                                           conv_desc,
                                           CUDNN_CONVOLUTION_FWD_ALGO_IMPLICIT_GEMM,
                                           nullptr, 0,
                                           &beta,
                                           hidden1_desc, d_hidden1));
        CHECK_CUDNN(cudnnAddTensor(cudnn, &alpha, hidden1_desc, d_b1,
                                  &alpha, hidden1_desc, d_hidden1));
        CHECK_CUDNN(cudnnActivationForward(cudnn, relu_desc,
                                          &alpha, hidden1_desc, d_hidden1,
                                          &beta, hidden1_desc, d_hidden1));

        // Layer 2: Hidden1 -> Hidden2
        CHECK_CUDNN(cudnnConvolutionForward(cudnn, &alpha,
                                           hidden1_desc, d_hidden1,
                                           fc2_w_desc, d_w2,
                                           conv_desc,
                                           CUDNN_CONVOLUTION_FWD_ALGO_IMPLICIT_GEMM,
                                           nullptr, 0,
                                           &beta,
                                           hidden2_desc, d_hidden2));
        CHECK_CUDNN(cudnnAddTensor(cudnn, &alpha, hidden2_desc, d_b2,
                                  &alpha, hidden2_desc, d_hidden2));
        CHECK_CUDNN(cudnnActivationForward(cudnn, relu_desc,
                                          &alpha, hidden2_desc, d_hidden2,
                                          &beta, hidden2_desc, d_hidden2));

        // Layer 3: Hidden2 -> Output
        CHECK_CUDNN(cudnnConvolutionForward(cudnn, &alpha,
                                           hidden2_desc, d_hidden2,
                                           fc3_w_desc, d_w3,
                                           conv_desc,
                                           CUDNN_CONVOLUTION_FWD_ALGO_IMPLICIT_GEMM,
                                           nullptr, 0,
                                           &beta,
                                           output_desc, d_output));
        CHECK_CUDNN(cudnnAddTensor(cudnn, &alpha, output_desc, d_b3,
                                  &alpha, output_desc, d_output));

        // Cleanup intermediate buffers
        CHECK_CUDA(cudaFree(d_hidden1));
        CHECK_CUDA(cudaFree(d_hidden2));

        // Inference (example)
        if (epoch == epochs - 1) {
            float* h_output = new float[batch_size * output_size];
            CHECK_CUDA(cudaMemcpy(h_output, d_output, batch_size * output_size * sizeof(float),
                                 cudaMemcpyDeviceToHost));
            std::cout << "Final inference results sample: " << h_output[0] << std::endl;
            delete[] h_output;
        }
    }
    printf("Training loop finished.\n"); // Added printf

    // Cleanup
    printf("Cleaning up device memory and cuDNN resources...\n"); // Added printf
    CHECK_CUDA(cudaFree(d_input));
    CHECK_CUDA(cudaFree(d_labels));
    CHECK_CUDA(cudaFree(d_output));
    CHECK_CUDA(cudaFree(d_w1));
    CHECK_CUDA(cudaFree(d_b1));
    CHECK_CUDA(cudaFree(d_w2));
    CHECK_CUDA(cudaFree(d_b2));
    CHECK_CUDA(cudaFree(d_w3));
    CHECK_CUDA(cudaFree(d_b3));

    CHECK_CUDNN(cudnnDestroyConvolutionDescriptor(conv_desc));
    CHECK_CUDNN(cudnnDestroyActivationDescriptor(relu_desc));
    CHECK_CUDNN(cudnnDestroyFilterDescriptor(fc1_w_desc));
    CHECK_CUDNN(cudnnDestroyFilterDescriptor(fc2_w_desc));
    CHECK_CUDNN(cudnnDestroyFilterDescriptor(fc3_w_desc));
    CHECK_CUDNN(cudnnDestroyTensorDescriptor(input_desc));
    CHECK_CUDNN(cudnnDestroyTensorDescriptor(hidden1_desc));
    CHECK_CUDNN(cudnnDestroyTensorDescriptor(hidden2_desc));
    CHECK_CUDNN(cudnnDestroyTensorDescriptor(output_desc));
    CHECK_CUDNN(cudnnDestroy(cudnn));
    printf("Cleanup complete.\n"); // Added printf

    return 0;
}

Overwriting fcnet.cu


In [17]:
!nvcc fcnet.cu -o fcnet -gencode arch=compute_75,code=sm_75 -lcublas -lcudnn -lcurand

!./fcnet

Initializing CUDA and cuDNN...
CUDA and cuDNN initialized.
Creating tensor descriptors...
Tensor descriptors created.
Setting tensor dimensions...
Tensor dimensions set.
Creating filter descriptors...
Filter descriptors created.
Creating activation descriptor...
Activation descriptor created.
Allocating device memory...
Device memory allocated.
Initializing weights and biases...
Weights and biases initialized.
Generating dummy data...
Dummy data generated.
Creating convolution descriptor...
Convolution descriptor created.
Starting training loop for 10 epochs...
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Final inference results sample: 1.67509e+07
Training loop finished.
Cleaning up device memory and cuDNN resources...
Cleanup complete.
